Stopwords'ları çıkarmak

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import nltk
from nltk.corpus import stopwords
import string

# Yorumları yükle
reddit_comments = pd.read_csv("ekonomi_birlesim.csv")

# Türkçe stopwords listesi ve kendi stopwords listenizi ekleyin
turkish_stopwords = set([
    "acaba", "ama", "ancak", "artık", "aslında", "az", "bana", "bazen", "bazı", "belki", "ben", "benden", "beni", "benim", 
    "beri", "bir", "biraz", "birçok", "birşey", "biz", "bize", "bizden", "bizi", "bizim", "böyle", "böylece", "bu", "buna", 
    "bunda", "bundan", "bunu", "bunun", "burada", "bütün", "çoğu", "çoğunu", "çok", "çünkü", "da", "daha", "dahi", "de", 
    "defa", "diye", "dolayı", "elbette", "en", "fakat", "falan", "filan", "gibi", "gene", "gereği", "göre", "hala", "hangi", 
    "hani", "hatta", "hem", "henüz", "hep", "hepsi", "her", "herhangi", "herkes", "herkese", "herkesi", "herkesin", "hiç", 
    "hiçbir", "için", "iki", "ile", "ilgili", "ise", "işte", "itibaren", "kaç", "kadar", "karşın", "kendi", "kendine", 
    "kendini", "kendisi", "kez", "ki", "kim", "kime", "kimi", "kimin", "kimisi", "kimse", "kısaca", "kadar", "mı", "mi", 
    "mu", "mü", "nasıl", "ne", "neden", "nedenle", "nerde", "nerede", "nereye", "neyse", "niçin", "niye", "o", "öyle", 
    "oldu", "olduğu", "olduğunu", "olarak", "olduktan", "olmadı", "olmadığı", "olmak", "olması", "olmayan", "olmaz", 
    "olsa", "olsun", "olup", "olur", "olursa", "oluyor", "onlar", "onlardan", "onları", "onların", "onu", "onun", "orada", 
    "öte", "ötürü", "öyle", "pek", "rağmen", "sadece", "sanki", "şayet", "şekilde", "şimdi", "siz", "sizden", "sizi", 
    "sizin", "sonra", "şöyle", "şu", "şuna", "şunda", "şundan", "şunlar", "şunu", "şunun", "ta", "tabii", "tam", "tamam", 
    "tamamen", "tarafından", "trilyon", "tüm", "üç", "üzere", "var", "vardı", "ve", "veya", "veyahut", "ya", "yani", 
    "yapacak", "yapılan", "yapılması", "yapıyor", "yapmak", "yaptı", "yaptığı", "yaptığını", "yaptıkları", "yedi", "yerine", 
    "yetmiş", "yine", "yirmi", "yoksa", "yüz", "zaten", "zira", "küfür1", "küfür2", "küfür3", "küfür4", "küfür5"
])

# Metin ön işleme
def preprocess_text(text):
    text = text.lower()  # Küçük harfe çevir
    text = text.translate(str.maketrans('', '', string.punctuation))  # Noktalama işaretlerini kaldır
    text = ' '.join([word for word in text.split() if word not in turkish_stopwords])  # Stop kelimeleri kaldır
    return text

reddit_comments['Processed_Comment'] = reddit_comments['Comment'].apply(preprocess_text)

# Özellik çıkarma (TF-IDF)
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(reddit_comments['Processed_Comment'])

# K-means algoritması ile kümeleme
kmeans = KMeans(n_clusters=5, random_state=42)
reddit_comments['Cluster'] = kmeans.fit_predict(X)

# Kullanıcı verilerini yükle
user_data = pd.read_csv("realistic_user_data.csv")

# Kullanıcıları yorum kümelerine göre atama
user_data['Comment'] = reddit_comments['Comment'].sample(n=len(user_data), replace=True).values
user_data['Comment_Cluster'] = reddit_comments['Cluster'].sample(n=len(user_data), replace=True).values

# Kümeleme sonuçlarını incele
print(user_data[['Age', 'Job', 'Depression', 'Comment', 'Comment_Cluster', 'Anxiety', 'Obesity', 'Active']].head(20))

# Veriyi CSV olarak kaydet
user_data.to_csv("clustered_user_data_with_comments.csv", index=False, encoding="utf-8-sig")
print("Kullanıcı verileri yorum kümeleri ile eşleştirildi ve kaydedildi.")

FileNotFoundError: [Errno 2] No such file or directory: 'reddit_comments.csv'

Önce küçük/büyük harf ve boşlukların kontrolünü sağlama 

In [1]:
import re
import pandas as pd

def preprocess_text(text):
    """
    Optimized text preprocessing function with compiled regex patterns
    and more efficient operations.
    """
    # Compile regex patterns once (more efficient)
    punctuation_pattern = re.compile(r"[,.:;!?()\"\'\[\]]")
    newline_pattern = re.compile(r"\n+")
    whitespace_pattern = re.compile(r"\s+")
    
    # Chain operations efficiently
    text = (text.lower()  # Convert to lowercase
           .strip())      # Remove leading/trailing whitespace
    
    # Apply compiled patterns
    text = punctuation_pattern.sub("", text)
    text = newline_pattern.sub(" ", text)
    text = whitespace_pattern.sub(" ", text)
    
    return text

# File handling with context manager
def process_file(input_file, output_file):
    try:
        # Read the input file
        df = pd.read_csv(input_file)
        
        # Process the text column (assuming it exists)
        if 'Body' in df.columns:
            df['cleaned_text'] = df['Body'].apply(preprocess_text)
        
        # Save to output file
        df.to_csv(output_file, index=False)
        return True
    except Exception as e:
        print(f"Error processing file: {str(e)}")
        return False

# Usage
input_file = "C:\\Users\\info\\OneDrive\\Masaüstü\\anket\\yorumlar\\ekonomi_birlesim.csv"
output_file = "ekonomi_cleaned2.csv"

if process_file(input_file, output_file):
    print("Text preprocessing completed successfully!")



Text preprocessing completed successfully!


In [9]:
import re
import pandas as pd

def preprocess_text(text):
    """
    Optimize edilmiş metin temizleme fonksiyonu: 
    - Noktalama işaretleri, fazla boşluklar ve yeni satır karakterleri temizlenir.
    """
    # NaN veya geçersiz metinleri kontrol et
    if pd.isna(text) or not isinstance(text, str):
        return ""
        
    # Regex desenleri (önceden derlenmiş)
    punctuation_pattern = re.compile(r"[,.:;!?()\"\'\[\]]")  # Noktalama işaretlerini kaldır
    newline_pattern = re.compile(r"\n+")                    # Yeni satır karakterlerini kaldır
    whitespace_pattern = re.compile(r"\s+")                 # Fazla boşlukları tek boşlukla değiştir
    
    # Metni temizleme işlemleri
    text = text.lower().strip()                # Küçük harf ve baştaki/sondaki boşluklar
    text = punctuation_pattern.sub("", text)   # Noktalama kaldır
    text = newline_pattern.sub(" ", text)      # Yeni satırları boşlukla değiştir
    text = whitespace_pattern.sub(" ", text)   # Fazla boşlukları tek boşlukla değiştir
    
    return text

def process_file(input_file, output_file):
    """
    Giriş dosyasındaki 'Body' sütununu temizler ve 'Metin' adında yeni sütun ekler.
    """
    try:
        # CSV dosyasını yükle
        df = pd.read_csv(input_file, encoding='utf-8')
        
        # Giriş dosyasında 'Body' sütunu kontrolü
        if 'Body' in df.columns:
            print("Metin sütunu işleniyor...")
            
            # Metin temizleme fonksiyonunu uygula ve 'Metin' adlı yeni sütuna yaz
            df['Metin'] = df['Body'].astype(str).apply(preprocess_text)
            
            # Sonuç dosyasını kaydet
            df.to_csv(output_file, index=False, encoding='utf-8')
            print(f"Metin temizleme işlemi tamamlandı! Toplam {len(df)} satır işlendi.")
            return True
        else:
            raise ValueError("Giriş dosyasında 'Body' adlı sütun bulunamadı!")
    
    except Exception as e:
        print(f"Hata oluştu: {str(e)}")
        return False

# Kullanım
input_file = "C:\\Users\\info\\OneDrive\\Masaüstü\\anket\\ekonomi2\\ekonomi_birlesim.csv"
output_file = "C:\\Users\\info\\OneDrive\\Masaüstü\\anket\\ekonomi2\\ekonomi_cleaned.csv"

if process_file(input_file, output_file):
    print(f"Temizlenmiş metinler '{output_file}' dosyasına başarıyla kaydedildi.")



Metin sütunu işleniyor...
Metin temizleme işlemi tamamlandı! Toplam 1538 satır işlendi.
Temizlenmiş metinler 'C:\Users\info\OneDrive\Masaüstü\anket\ekonomi2\ekonomi_cleaned.csv' dosyasına başarıyla kaydedildi.


Stopwordsleri çıkarma işlemleri

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import nltk
from nltk.corpus import stopwords
import string

# Yorumları yükle
reddit_comments = pd.read_csv("reddit_comments.csv")

# Metin ön işleme
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def preprocess_text(text):
    text = text.lower()  # Küçük harfe çevir
    text = text.translate(str.maketrans('', '', string.punctuation))  # Noktalama işaretlerini kaldır
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Stop kelimeleri kaldır
    return text

reddit_comments['Processed_Comment'] = reddit_comments['Comment'].apply(preprocess_text)

In [ ]:


# Özellik çıkarma (TF-IDF)
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(reddit_comments['Processed_Comment'])

# K-means algoritması ile kümeleme
kmeans = KMeans(n_clusters=5, random_state=42)
reddit_comments['Cluster'] = kmeans.fit_predict(X)

# Kullanıcı verilerini yükle
user_data = pd.read_csv("realistic_user_data.csv")

# Kullanıcıları yorum kümelerine göre atama
user_data['Comment'] = reddit_comments['Comment'].sample(n=len(user_data), replace=True).values
user_data['Comment_Cluster'] = reddit_comments['Cluster'].sample(n=len(user_data), replace=True).values

# Kümeleme sonuçlarını incele
print(user_data[['Age', 'Job', 'Depression', 'Comment', 'Comment_Cluster', 'Anxiety', 'Obesity', 'Active']].head(20))

# Veriyi CSV olarak kaydet
user_data.to_csv("clustered_user_data_with_comments.csv", index=False, encoding="utf-8-sig")
print("Kullanıcı verileri yorum kümeleri ile eşleştirildi ve kaydedildi.")